In [1]:
import numpy as np
import pandas as pd

In [2]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
movies.shape

(9742, 3)

In [5]:
combine_movies_rating = pd.merge(ratings, movies, on='movieId')
columns = ['genres', 'timestamp']
combine_movies_rating = combine_movies_rating.drop(columns, axis=1)
combine_movies_rating.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [6]:
movie_ratingCount = (combine_movies_rating.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )
movie_ratingCount.head()

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [7]:
rating_with_totalRatingCount = combine_movies_rating.merge(movie_ratingCount) 
rating_with_totalRatingCount.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


popularity_threshold = 50
rating_popular_movie = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_movie.tail()

In [8]:
rating_with_totalRatingCount = rating_with_totalRatingCount.drop_duplicates(['userId', 'title'])

In [9]:
user_rating_pivot2 = rating_with_totalRatingCount.pivot(index = 'userId', columns = 'title', values = 'rating').fillna(0)
user_rating_pivot2.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
user_rating_pivot2.shape

(610, 9719)

In [22]:
X = user_rating_pivot2.values.T
X.shape

(9719, 610)

In [23]:
import sklearn
from sklearn.decomposition import TruncatedSVD

SVD = TruncatedSVD(n_components=12, random_state=17)
matrix = SVD.fit_transform(X)
matrix.shape

(9719, 12)

In [24]:
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)
corr = np.corrcoef(matrix)
corr.shape

(9719, 9719)

In [14]:
movie_title = user_rating_pivot2.columns
movie_list = list(movie_title)
X_men = movie_list.index("Waiting to Exhale (1995)")
print(X_men)

9225


In [21]:
corr_X_men  = corr[X_men]
list(movie_title[(corr_X_men >= 0.91)])

['Adventures of Pinocchio, The (1996)',
 'Babysitter, The (1995)',
 'Before and After (1996)',
 'Carpool (1996)',
 'Cure, The (1995)',
 'Dangerous Ground (1997)',
 'Forget Paris (1995)',
 'Gordy (1995)',
 'I Like It Like That (1994)',
 "It's My Party (1996)",
 'Jefferson in Paris (1995)',
 'Losing Isaiah (1995)',
 'Mr. Wrong (1996)',
 'My Family (1995)',
 'Nothing Personal (1995)',
 'Perez Family, The (1995)',
 'Poetic Justice (1993)',
 'Reckless (1995)',
 'Rent-a-Kid (1995)',
 'Serial Mom (1994)',
 'Simple Twist of Fate, A (1994)',
 'Spirits of the Dead (1968)',
 'Stupids, The (1996)',
 'Three Wishes (1995)',
 'Touch (1997)',
 'Waiting to Exhale (1995)',
 'Walking Dead, The (1995)']

In [16]:
def recommend(movie):
    movie_title = user_rating_pivot2.columns
    movie_list = list(movie_title)
    X = movie_list.index(movie)
    #print(X_men)
    corr_of_movie  = corr[X]
    return list(movie_title[(corr_of_movie >= 0.9)])

In [17]:
recommend("It's My Party (1996)")

['Adventures of Pinocchio, The (1996)',
 'Babysitter, The (1995)',
 'Before and After (1996)',
 'Beyond Bedlam (1993)',
 'Carpool (1996)',
 'Castle Freak (1995)',
 'Children of the Corn IV: The Gathering (1996)',
 'Cure, The (1995)',
 'Dangerous Ground (1997)',
 'Disclosure (1994)',
 'Forget Paris (1995)',
 'French Kiss (1995)',
 'Frisk (1995)',
 'Gordy (1995)',
 'Houseguest (1994)',
 'I Like It Like That (1994)',
 "It's My Party (1996)",
 'Jefferson in Paris (1995)',
 'Lassie (1994)',
 'Losing Isaiah (1995)',
 'Love Affair (1994)',
 'Man of the House (1995)',
 'Man of the Year (1995)',
 'Mr. Wrong (1996)',
 'My Family (1995)',
 'NeverEnding Story III, The (1994)',
 "Nobody's Fool (1994)",
 'Nothing Personal (1995)',
 'Perez Family, The (1995)',
 'Poetic Justice (1993)',
 'Reckless (1995)',
 'Relative Fear (1994)',
 'Rent-a-Kid (1995)',
 'Simple Twist of Fate, A (1994)',
 'Specialist, The (1994)',
 'Spirits of the Dead (1968)',
 'Stupids, The (1996)',
 'Three Wishes (1995)',
 'Touch (1